# Pedram ShahSafi

# pd.shahsafi@gmail.com

<img src="pics/1.jpg" />

# Define and Access the Database

The application will use a SQLite database to store users and posts. Python comes with built-in support for SQLite in the sqlite3 module.

SQLite is convenient because it doesn’t require setting up a separate database server and is built-in to Python. However, if concurrent requests try to write to the database at the same time, they will slow down as each write happens sequentially. Small applications won’t notice this. Once you become big, you may want to switch to a different database.

# Connect to the Database

The first thing to do when working with a SQLite database (and most other Python database libraries) is to create a connection to it. Any queries and operations are performed using the connection, which is closed after the work is finished.

In web applications this connection is typically tied to the request. It is created at some point when handling a request, and closed before the response is sent.

In [ ]:
(flaskEnv) pd@asghar:~/project/flaskr$ touch db.py

# db.py

In [ ]:
import sqlite3

import click
from flask import current_app, g
from flask.cli import with_appcontext


def get_db():
    if 'db' not in g:
        g.db = sqlite3.connect(
            current_app.config['DATABASE'],
            detect_types=sqlite3.PARSE_DECLTYPES
        )
        g.db.row_factory = sqlite3.Row

    return g.db


def close_db(e=None):
    db = g.pop('db', None)

    if db is not None:
        db.close()

## g 
1. g is a special object that is unique for each request.
2. It is used to store data that might be accessed by multiple functions during the request.
3. The connection is stored and reused instead of creating a new connection if get_db is called a second time in the same request.


## current_app
1. current_app is another special object that points to the Flask application handling the request.
2. Since you used an application factory, there is no application object when writing the rest of your code. 3. get_db will be called when the application has been created and is handling a request, so current_app can be used.

## sqlite3.connect()
1. sqlite3.connect() establishes a connection to the file pointed at by the DATABASE configuration key.
2. This file doesn’t have to exist yet, and won’t until you initialize the database later.
4. sqlite3.PARSE_DECLTYPES This constant is meant to be used with the detect_types parameter of the connect() function. Setting it makes the sqlite3 module parse the declared type for each column it returns. It will parse out the first word of the declared type, i. e. for “integer primary key”, it will parse out “integer”, or for “number(10)” it will parse out “number”. Then for that column, it will look into the converters dictionary and use the converter function registered for that type there.

## sqlite3.Row 
1. tells the connection to return rows that behave like dicts. This allows accessing the columns by name.

## close_db 
1. close_db checks if a connection was created by checking if g.db was set.
2. If the connection exists, it is closed.
3. Further down you will tell your application about the close_db function in the application factory so that it is called after each request.

## Click
click is a Python package for creating beautiful command line interfaces in a composable way with as little code as necessary. It’s the “Command Line Interface Creation Kit”. It’s highly configurable but comes with sensible defaults out of the box.

In [ ]:
import click

@click.command()

@click.option(
    '--count',
    default=1,
    help='Number of greetings.'
)

@click.option(
    '--name',
    prompt='Your name',
    help='The person to greet.'
)

def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for x in range(count):
        click.echo('Hello %s!' % name)

if __name__ == '__main__':
    hello()


In [ ]:
$ python hello.py --count=3
Your name: John
Hello John!
Hello John!
Hello John!

In [ ]:
$ python hello.py --help
Usage: hello.py [OPTIONS]

  Simple program that greets NAME for a total of COUNT times.

Options:
  --count INTEGER  Number of greetings.
  --name TEXT      The person to greet.
  --help           Show this message and exit.

# Create the Tables

In SQLite, data is stored in tables and columns. These need to be created before you can store and retrieve data. Flaskr will store users in the user table, and posts in the post table. Create a file with the SQL commands needed to create empty tables:

In [ ]:
(flaskEnv) pd@asghar:~/project/flaskr$ touch schema.sql


## flaskr/schema.sql

In [ ]:
DROP TABLE IF EXISTS user;
DROP TABLE IF EXISTS post;

CREATE TABLE user (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  username TEXT UNIQUE NOT NULL,
  password TEXT NOT NULL
);

CREATE TABLE post (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  author_id INTEGER NOT NULL,
  created TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
  title TEXT NOT NULL,
  body TEXT NOT NULL,
  FOREIGN KEY (author_id) REFERENCES user (id),
);

Add the Python functions that will run these SQL commands to the db.py file:

## flaskr/db.py

In [ ]:
def init_db():
    db = get_db()

    with current_app.open_resource('schema.sql') as f:
        db.executescript(f.read().decode('utf8'))


@click.command('init-db')
@with_appcontext
def init_db_command():
    """Clear the existing data and create new tables."""
    init_db()
    click.echo('Initialized the database.')

### open_resource()
1. opens a file relative to the flaskr package,
2. which is useful since you won’t necessarily know where that location is when deploying the application later.
3. get_db returns a database connection, which is used to execute the commands read from the file.

### click.command() 
1. defines a command line command called init-db that calls the init_db function and shows a success message to the user.


#  Register with the Application

The close_db and init_db_command functions need to be registered with the application instance, otherwise they won’t be used by the application. However, since you’re using a factory function, that instance isn’t available when writing the functions. Instead, write a function that takes an application and does the registration.

### flaskr/db.py

In [ ]:
def init_app(app):
    app.teardown_appcontext(close_db)
    app.cli.add_command(init_db_command)

### app.teardown_appcontext()

1. tells Flask to call that function when cleaning up after returning the response.

### app.cli.add_command() 
1. adds a new command that can be called with the flask command.

Import and call this function from the factory. Place the new code at the end of the factory function before returning the app.

### flaskr/__init__.py

In [ ]:
def create_app():
    app = ...
    # existing code omitted

    from . import db
    db.init_app(app)

    return app

# Initialize the Database File

Now that init-db has been registered with the app, it can be called using the flask command.

In [ ]:
flask init-db
Initialized the database.